In [1]:
import datetime

In [2]:
MODEL_NAME = "fulltime_win_outcome"
MODEL_VERSION = 1.0

In [3]:
# UPDATING THE GITHUB OF THE DATASETS
MODEL_TIME = datetime.datetime.now().strftime("%d-%M-%Y_%T")
!git add -A
!git commit -m "updates: {MODEL_NAME}_v{MODEL_VERSION}: bot commit - {MODEL_TIME}"
!git push

[main c3dfbc9] updates: fulltime_win_outcome_v1.0: bot commit - 27-46-2024_08:46:00
 6 files changed, 30525 insertions(+), 52 deletions(-)
 rewrite models/models-registry.json (85%)
 create mode 100644 relative_datasets/cleaned/fulltime_win_outcome_v1.0-test-set.csv
 create mode 100644 relative_datasets/cleaned/fulltime_win_outcome_v1.0-train-set.csv
 create mode 100644 relative_datasets/raw/odds_training.csv
Enumerating objects: 22, done.
Counting objects: 100% (22/22), done.
Delta compression using up to 2 threads
Compressing objects: 100% (13/13), done.
Writing objects: 100% (13/13), 282.70 KiB | 1.08 MiB/s, done.
Total 13 (delta 8), reused 0 (delta 0), pack-reused 0
remote: Resolving deltas: 100% (8/8), completed with 7 local objects.
To https://github.com/lebyanelm/neural-trained-models.git
   3805b17..c3dfbc9  main -> main
